In [2]:
import numpy as np

def read_obj_vertices(filename):
    vertices = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertex = np.array(parts[1:4], dtype=float)
                vertices.append(vertex)
    return np.array(vertices)

def calculate_average_vertex_error(obj_file1, obj_file2):
    vertices1 = read_obj_vertices(obj_file1)
    vertices2 = read_obj_vertices(obj_file2)
    
    if len(vertices1) != len(vertices2):
        raise ValueError("顶点数量不匹配")
    
    errors = np.linalg.norm(vertices1 - vertices2, axis=1)
    average_error = np.mean(errors)
    return average_error

# 替换为您的文件路径
obj_file1 = '0001_garment.obj'
obj_file2 = '0000_garment.obj'

average_error = calculate_average_vertex_error(obj_file1, obj_file2)
print(f"每顶点平均误差: {average_error}")


每顶点平均误差: 0.010047981904609425


In [4]:
import numpy as np

def read_obj(filepath):
    vertices = []
    faces = []
    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith('v '):
                vertex = list(map(float, line.strip().split(' ')[1:]))
                vertices.append(vertex)
            elif line.startswith('f '):
                face = list(map(lambda x: int(x) - 1, line.strip().split(' ')[1:]))
                faces.append(face)
    return np.array(vertices), np.array(faces)

def detect_collision(vertex, vertices, faces):
    # 碰撞检测的简单示例：检查顶点是否在任何一个面片的包围盒内
    for face in faces:
        face_vertices = vertices[face]
        min_coords = np.min(face_vertices, axis=0)
        max_coords = np.max(face_vertices, axis=0)
        if np.all(vertex >= min_coords) and np.all(vertex <= max_coords):
            return True
    return False

# 读取服装和障碍物的OBJ文件
garment_vertices, garment_faces = read_obj('0046_garment.obj')
obstacle_vertices, obstacle_faces = read_obj('0046_obstacle.obj')

# 初始化碰撞计数
collision_count = 0

# 对每个服装顶点进行碰撞检测
for vertex in garment_vertices:
    if detect_collision(vertex, obstacle_vertices, obstacle_faces):
        collision_count += 1

# 计算平均碰撞次数
average_collisions = collision_count / len(garment_vertices)

print(f'Average number of collisions per vertex: {average_collisions}')


Average number of collisions per vertex: 0.07840564258180924


In [7]:
import numpy as np

def read_obj(filepath):
    vertices = []
    faces = []
    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith('v '):
                vertex = list(map(float, line.strip().split(' ')[1:]))
                vertices.append(vertex)
            elif line.startswith('f '):
                face = list(map(lambda x: int(x.split('/')[0]) - 1, line.strip().split(' ')[1:]))
                faces.append(face)
    return np.array(vertices), np.array(faces)

def detect_self_collision(vertices, faces):
    collision_count = 0
    for i, vertex in enumerate(vertices):
        for face in faces:
            # 确保顶点不在它所属的面片上
            if i not in face:
                # 简单的碰撞检测：检查顶点是否在面片的包围盒内
                face_vertices = vertices[face]
                min_coords = np.min(face_vertices, axis=0)
                max_coords = np.max(face_vertices, axis=0)
                if np.all(vertex >= min_coords) and np.all(vertex <= max_coords):
                    collision_count += 1
                    break  # 一个顶点只计算一次碰撞
    return collision_count

# 读取服装模型的OBJ文件
garment_vertices, garment_faces = read_obj('0001_garment.obj')

# 进行自碰撞检测
total_collisions = detect_self_collision(garment_vertices, garment_faces)

# 计算每顶点的平均碰撞次数
average_collisions = total_collisions / len(garment_vertices)

print(f'Average number of self-collisions per vertex: {average_collisions}')


Average number of self-collisions per vertex: 0.023481664009276707
